### WEB SCRAPPING ASSIGNMENT 3

In [11]:
from selenium import webdriver
import time
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
import warnings
import requests
warnings.filterwarnings('ignore')
from selenium.webdriver.common.by import By
import pandas as pd 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import regex as re

1. Write a python program which searches all the product under a particular product from www.amazon.in. The
product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars

In [60]:
driver=webdriver.Chrome()
driver.get("http://www.amazon.in/")

search = driver.find_element(By.ID, "twotabsearchtextbox")

item = str(input("Enter item name that you want to search: "))
search.send_keys(item)

search_btn=driver.find_element(By.ID, "nav-search-submit-button")
search_btn.click()
time.sleep(5)


Enter item name that you want to search: guitar


2. In the above question, now scrape the following details of each product listed in first 3 pages of your search
results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then
scrape all the products available under that product name. Details to be scraped are: "Brand
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [61]:
urls =[]
for i in range(0,3):
    for x in driver.find_elements(By.CSS_SELECTOR, "a.a-link-normal.s-no-outline"):
        urls.append(x.get_attribute('href'))
    driver.find_element(By.LINK_TEXT, "Next").click()
    time.sleep(3)
brand=[]
prod_name=[]
rating=[]
rating_num=[]
price= []
ret_ex=[]
exp_del = []
availability=[]
prod_url=[]
ot_det = []

for i in urls[:10]:
    driver.get(i)
    time.sleep(3)

    try:
        brand_name=driver.find_element(By.XPATH, "//td[@class='a-span9']/span")
        brand.append(brand_name.text)
    except NoSuchElementException:
        brand.append('-')    
            
    try:
        product=driver.find_element(By.ID, "productTitle")
        prod_name.append(product.text)
    except NoSuchElementException:
        prod_name.append('-')         
        
    try:
        ratestar=driver.find_element(By.XPATH, "//span[@class='a-size-base a-nowrap']//span")
        rating.append(ratestar.text)
    except NoSuchElementException:
        rating.append("No Rating")       
                
    try:
        ratenum=driver.find_element(By.ID, "acrCustomerReviewText")
        rating_num.append(ratenum.text)
    except NoSuchElementException:
        rating_num.append('-')         
                
    try:
        prod_price=driver.find_element(By.CLASS_NAME, "priceToPay")
        price.append(prod_price.text)
    except NoSuchElementException:
        price.append('-')      
                
    try:
        ret_exch=driver.find_element(By.XPATH, "//*[@id='RETURNS_POLICY']/span/div[2]/span")
        ret_ex.append(ret_exch.text)
    except NoSuchElementException:
        ret_ex.append('-') 
        
    try:
        ex_de=driver.find_element(By.ID, "mir-layout-DELIVERY_BLOCK-slot-PRIMARY_DELIVERY_MESSAGE_LARGE")
        exp_del.append(ex_de.text)
    except NoSuchElementException:
        exp_del.append('-')           
               
    try:
        availa=driver.find_element(By.ID, "availability")
        availability.append(availa.text)
    except NoSuchElementException:
        availability.append('-')       
                           
    prod_url.append(i)
        
    try:
        othr_det=driver.find_element(By.XPATH, "//ul[@class='a-unordered-list a-vertical a-spacing-mini']/li/span")
        ot_det.append(othr_det.text)
    except NoSuchElementException:
        ot_det.append('-')
        
item_df = pd.DataFrame({})
item_df['Brand Name'] = brand
item_df['Product Name'] = prod_name
item_df['Rating'] = rating
item_df['Rating Number'] = rating_num
item_df['Price'] = price
item_df['Return/Exchange'] = ret_ex
item_df['Expected Delivery'] = exp_del
item_df['Availability'] = availability
item_df['Product URLs'] = prod_url
item_df['Other Details'] = ot_det     
item_df

In [ ]:
item_df.to_csv(item + '.csv', index=False)  

3. Write a python program to access the search bar and search button on images.google.com and scrape 10
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cake

In [4]:
driver=webdriver.Chrome()
def fetch_google_images(searchStr):
    url='https://images.google.com/'
    driver.get(url)
    time.sleep(3)
    search_bar = driver.find_element(By.ID, "APjFqb")   
    search_bar.send_keys(str(searchStr))
    search_bar.submit()   
    img_url = []

    while len(img_url) <= 10:
        driver.execute_script("window.scrollBy(0,100)")        
        imgs = driver.find_elements(By.CSS_SELECTOR, ".rg_i.Q4LuWd")
        for image in imgs:
            source = image.get_attribute('src')
            if source is not None and source.startswith('http'):
                img_url.append(source)                
    for i, img_src in enumerate(img_url[:10]):  
        response=requests.get(img_src)
        file = open(r"C:\\data\\"+searchStr+"-"+str(i)+".jpg", "wb")
        file.write(response.content)
     

fetch_google_images("Fruits")
fetch_google_images("cars")
fetch_google_images("Machine Learning")
fetch_google_images("Guitar")
fetch_google_images("Cakes")
  

4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com
and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand
Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the
details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [60]:
driver=webdriver.Chrome()
driver.get("http://www.flipkart.com/")
search = driver.find_element(By.CLASS_NAME, 'Pke_EE')
search.send_keys("smartphone")
search.submit()

time.sleep(5)

In [61]:
urls = []
for i in driver.find_elements(By.XPATH, "//a[@class='_1fQZEK']"):
    urls.append(i.get_attribute("href"))

brand_names = []
name = []
color = []
RAM = []
storage = []
prim_cam = []
sec_cam = []
Display_size = []
display_resolution = []
processor = []
processor_core = []
battery_cap = []
prices = []
product_url = []

for i in driver.find_elements(By.XPATH, "//div[@class='fMghEO']"):
    try:
        bat_cap = driver.find_element(By.XPATH, "//ul[@class='_1xgFaf']//li[4]")
        battery_cap.append(bat_cap.text)
    except NoSuchElementException:
        battery_cap.append('-')
for i in urls:
    driver.get(i)
    time.sleep(3)

    try:
        br_name = driver.find_element(By.XPATH, "/html/body/div/div/div[3]/div[1]/div[2]/div[1]/div[1]/div/div[4]/a")
        brand_names.append(br_name.text.replace('Mobiles',''))
    except NoSuchElementException:
        brand_names.append('-')

    try:
        smt_name = driver.find_element(By.XPATH, "//h1[@class='yhB1nd']//span")
        name.append(smt_name.text)
    except NoSuchElementException:
        name.append('-')
    
    try:
        clr = driver.find_element(By.XPATH, "//table[@class='_14cfVK']//tr[4]//td[2]")
        color.append(clr.text)
    except NoSuchElementException:
        color.append('-')
    time.sleep(2)
    
    try:
        read_more_btn = driver.find_element(By.XPATH, "//button[@class='_2KpZ6l _1FH0tX']").click()
    except NoSuchElementException:
        pass
    time.sleep(3)
    
    try:
        disp_size = driver.find_element(By.XPATH, "//div[@class='_1UhVsV']//div[2]//tr[1]//td[2]")
        Display_size.append(disp_size.text)
    except NoSuchElementException:
        Display_size.append
        
    try:
        disp_res = driver.find_element(By.XPATH, "//div[@class='_1UhVsV']//div[2]//tr[2]//td[2]")
        display_resolution.append(disp_res.text)
    except NoSuchElementException:
        display_resolution.append('-')
        
    try:
        pro = driver.find_element(By.XPATH, "//div[@class='_1UhVsV']//div[3]//tr[2]//td[2]")
        processor.append(pro.text)
    except NoSuchElementException:
        processor.append('-')
        
    try:
        pro_core = driver.find_element(By.XPATH, "//div[@class='_1UhVsV']//div[3]//tr[3]//td[2]")
        processor_core.append(pro_core.text)
    except NoSuchElementException:
        processor_core.append('-')
        
    try:
        rom = driver.find_element(By.XPATH, "//div[@class='_1UhVsV']//div[4]//tr[1]//td[2]")
        storage.append(rom.text)
    except NoSuchElementException:
        storage.append('-')
        
    try:
        ram = driver.find_element(By.XPATH, "//div[@class='_1UhVsV']//div[4]//tr[2]//td[2]")
        RAM.append(ram.text)
    except NoSuchElementException:
        RAM.append('-')
    time.sleep(2)
    
    try:
        p_cam = driver.find_element(By.XPATH, "//div[@class='_1UhVsV']//div[5]//tr[2]//td[2]")
        prim_cam.append(p_cam.text)
    except NoSuchElementException:
        prim_cam.append('-')
        
    try:
        s_cam = driver.find_element(By.XPATH, "//div[@class='_1UhVsV']//div[5]//tr[5]//td[2]")
        sec_cam.append(s_cam.text)
    except NoSuchElementException:
        sec_cam.append('-')        
   
    try:
        price = driver.find_element(By.XPATH, "//div[@class='_30jeq3 _16Jk6d']")
        prices.append(price.text.replace('₹','Rs. '))
    except NoSuchElementException:
        prices.append('-')
    
    
data = list(zip(brand_names,name,color,RAM,storage,prim_cam,sec_cam,Display_size,display_resolution,processor,
                processor_core,battery_cap,prices,urls))
df = pd.DataFrame(data, columns = ["Brand Name","Smartphone Name","Color","RAM","Storage/ROM","Primary Camera",
                                   "Secondary Camera","Display Size","Display Resolution","Processor","Processor-Core",
                                   "Battery Capacity","Price","Product Url"])
df
 

,Brand Name,Smartphone Name,Color,RAM,Storage/ROM,Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor-Core,Battery Capacity,Price,Product Url
0,vivo,"vivo T2x 5G (Marine Blue, 128 GB) (6 GB RAM)",Marine Blue,6 GB,128 GB,50MP + 2MP,8MP Front Camera,16.71 cm (6.58 inch),2408 x 1080 Pixels,Mediatek,Dimensity 6020,5000 mAh Battery,"Rs. 12,999",https://www.flipkart.com/vivo-t2x-5g-marine-bl...
1,vivo,"vivo T2x 5G (Glimmer Black, 128 GB) (6 GB RAM)",Glimmer Black,6 GB,128 GB,50MP + 2MP,8MP Front Camera,16.71 cm (6.58 inch),2408 x 1080 Pixels,Mediatek,Dimensity 6020,5000 mAh Battery,"Rs. 12,999",https://www.flipkart.com/vivo-t2x-5g-glimmer-b...
2,vivo,"vivo T2x 5G (Aurora Gold, 128 GB) (6 GB RAM)",Aurora Gold,6 GB,128 GB,50MP + 2MP,8MP Front Camera,16.71 cm (6.58 inch),2408 x 1080 Pixels,Mediatek,Dimensity 6020,5000 mAh Battery,"Rs. 12,999",https://www.flipkart.com/vivo-t2x-5g-aurora-go...
3,vivo,"vivo T2x 5G (Aurora Gold, 128 GB) (4 GB RAM)",Aurora Gold,4 GB,128 GB,50MP + 2MP,8MP Front Camera,16.71 cm (6.58 inch),2408 x 1080 Pixels,Mediatek,Dimensity 6020,5000 mAh Battery,"Rs. 11,999",https://www.flipkart.com/vivo-t2x-5g-aurora-go...
4,vivo,"vivo T2x 5G (Glimmer Black, 128 GB) (4 GB RAM)",Glimmer Black,4 GB,128 GB,50MP + 2MP,8MP Front Camera,16.71 cm (6.58 inch),2408 x 1080 Pixels,Mediatek,Dimensity 6020,5000 mAh Battery,"Rs. 11,999",https://www.flipkart.com/vivo-t2x-5g-glimmer-b...
5,vivo,"vivo T2x 5G (Aurora Gold, 128 GB) (8 GB RAM)",Aurora Gold,8 GB,128 GB,50MP + 2MP,8MP Front Camera,16.71 cm (6.58 inch),2408 x 1080 Pixels,Mediatek,Dimensity 6020,5000 mAh Battery,"Rs. 14,999",https://www.flipkart.com/vivo-t2x-5g-aurora-go...
6,vivo,"vivo T2x 5G (Marine Blue, 128 GB) (8 GB RAM)",Marine Blue,8 GB,128 GB,50MP + 2MP,8MP Front Camera,16.71 cm (6.58 inch),2408 x 1080 Pixels,Mediatek,Dimensity 6020,5000 mAh Battery,"Rs. 14,999",https://www.flipkart.com/vivo-t2x-5g-marine-bl...
7,vivo,"vivo T2x 5G (Marine Blue, 128 GB) (4 GB RAM)",Marine Blue,4 GB,128 GB,50MP + 2MP,8MP Front Camera,16.71 cm (6.58 inch),2408 x 1080 Pixels,Mediatek,Dimensity 6020,5000 mAh Battery,"Rs. 11,999",https://www.flipkart.com/vivo-t2x-5g-marine-bl...
8,vivo,"vivo T2x 5G (Glimmer Black, 128 GB) (8 GB RAM)",Glimmer Black,8 GB,128 GB,50MP + 2MP,8MP Front Camera,16.71 cm (6.58 inch),2408 x 1080 Pixels,Mediatek,Dimensity 6020,5000 mAh Battery,"Rs. 14,999",https://www.flipkart.com/vivo-t2x-5g-glimmer-b...
9,POCO,"POCO C51 (Royal Blue, 64 GB) (4 GB RAM)",Royal Blue,4 GB,64 GB,8MP Dual Rear Camera,5MP Front Camera,16.56 cm (6.52 inch),1600 x 720 Pixels,Mediatek,Helio G36,5000 mAh Battery,"Rs. 5,499",https://www.flipkart.com/poco-c51-royal-blue-6...


In [ ]:
df.to_csv('smartphone.csv', index=False) 

7. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped:
“Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [115]:
driver=webdriver.Chrome()
url = 'https://www.forbes.com/'
driver.get(url)
time.sleep(3)

optn = driver.find_element(By.CLASS_NAME, "_69hVhdY4")
optn.click()
time.sleep(1)

bill = driver.find_element(By.LINK_TEXT, "Billionaires")
a = ActionChains(driver)
a.move_to_element(bill).perform()


bill = driver.find_element(By.LINK_TEXT, "World's Billionaires")
bill.click()
time.sleep(1)
rank = []
names = []
net_worth = []
age = []
citizenship = []
source = []
industry = []

world_bill_row = driver.find_elements(By.CLASS_NAME, "TableRow_row__L-0Km")
for billionair in world_bill_row:
    cols = billionair.find_elements(By.CLASS_NAME, "TableRow_cell__db-hv")
    rank.append(cols[0].text)
    names.append(cols[1].text)
    net_worth.append(cols[2].text)
    age.append(cols[3].text)
    citizenship.append(cols[4].text)
    source.append(cols[5].text)
driver.find_elements(By.CSS_SELECTOR, ".Pagination_bubbleArrow__WFrX4.Pagination_paginationBtnNext__IOwqm")
        
        
data = list(zip(rank,names,net_worth,age,citizenship,source))
df = pd.DataFrame(data, columns = ["Rank", "Name", "Net worth", "Age", "Citizenship", "Source"])
df
        
        
        

,Rank,Name,Net worth,Age,Citizenship,Source
0,1,Bernard Arnault & family,$211 B,74,France,LVMH
1,2,Elon Musk,$180 B,51,United States,"Tesla, SpaceX"
2,3,Jeff Bezos,$114 B,59,United States,Amazon
3,4,Larry Ellison,$107 B,78,United States,Oracle
4,5,Warren Buffett,$106 B,92,United States,Berkshire Hathaway
...,...,...,...,...,...,...
195,195,Jin Baofang,$9.6 B,70,China,Solar panels
196,195,Luo Liguo & family,$9.6 B,67,China,Chemicals
197,195,Marijke Mars,$9.6 B,58,United States,"Candy, pet food"
198,195,Pamela Mars,$9.6 B,62,United States,"Candy, pet food"


8. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted
from any YouTube Video

In [53]:
driver = webdriver.Chrome()
url = "https://www.youtube.com/"
driver.get(url)
time.sleep(3)
search_bar = driver.find_element(By.XPATH, "//div[@class='ytd-searchbox-spt']/input")  
driver.implicitly_wait(5)
item = str(input("Enter what you want to search on YouTube: "))
search_bar.send_keys(item)
search_bar.submit()


time.sleep(3)

video = driver.find_element(By.ID, "video-title").click()
time.sleep(10)

Enter what you want to search on YouTube: sia- unstoppable


In [54]:
comments = []
comment_time = []
Time = []
Likes = []
No_of_Likes = []

for _ in range(1500):
    driver.execute_script("window.scrollBy(0,100)")

cm = driver.find_elements(By.ID, "main")
for c in cm:
    cc = c.find_element(By.ID, "comment-content")
    if cc.text is None:
        comments.append("--")
    else:
        comments.append(cc.text)
    
    ct = c.find_element(By.XPATH, "//*[@id='header-author']/yt-formatted-string/a")
    if ct.text is None:
        comment_time.append("--")
    else:
        comment_time.append(ct.text)
        
        
    cl = c.find_element(By.ID, "vote-count-middle")
    if cl.text is None:
        No_of_Likes.append("--")
    else:
        No_of_Likes.append(cl.text)    
time.sleep(4)



YT=pd.DataFrame({'Comments':comments,
                'Time':comment_time,
                'Likes':No_of_Likes})
YT
Youtube = pd.DataFrame({})
Youtube['Comment'] = comments[:500]
Youtube['Comment Time'] = comment_time[:500]
Youtube['Comment Upvotes'] = No_of_Likes[:500]
Youtube

,Comment,Comment Time,Comment Upvotes
0,2024 listeners?,1 month ago,7.1K
1,I love Sia's voice. For several moments I wond...,1 month ago,27
2,My 6 year old sings this out loud when in the ...,1 month ago,21
3,I have never been more impacted by the sheer f...,1 month ago,9
4,Who is here with me listening to this powerful...,1 month ago,250
...,...,...,...
495,️Yo teamoooo,1 month ago,
496,It's 2k24 and this song is still just wow,1 month ago,
497,,1 month ago,
498,I miss these songs from Sia. She’s the best si...,1 month ago,64


5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps

In [74]:
driver = webdriver.Chrome()
url = "https://www.google.com/maps/"
driver.get(url)
time.sleep(3)
search_bar = driver.find_element(By.ID, "searchboxinput")  
driver.implicitly_wait(5)
item = str(input("Enter the city name: "))
search_bar.send_keys(item)
search_bar.submit()
time.sleep(3)

try:
    url_str = driver.current_url
    latitude_longitude = re.findall(r'@(.*)',url_str)
    if len(latitude_longitude):
        lat_lng_list = latitude_longitude[0].split(",")
        if len(lat_lng_list)>=2:
            latitude = lat_lng_list[0]
            longitude = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(latitude, longitude))
except Exception as e:
        print("Error: ", str(e))


Enter the city name: charlotte
Latitude = 35.3107968, Longitude = -80.7665664


6. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [44]:
driver=webdriver.Chrome()
driver.get("https://www.digit.in")
search = driver.find_element(By.ID, "woocommerce-product-search-field-0")
search.send_keys("Gaming Laptop")
search_btn=driver.find_element(By.CLASS_NAME, "btnsearch")
search_btn.click()
time.sleep(5)
urls = []
Laptop_Name = []
Operating_sys = []
Display = []
Processor = []
Memory = []
Storage_capacity = []
Price = []
for _ in range(0, 2):
    lap = driver.find_elements(By.CLASS_NAME, "product")
    for i in lap:
        la= i.find_element(By.CSS_SELECTOR, "h3.text-clamp > a")
        urls.append(la.get_attribute('href'))
    driver.find_element(By.CSS_SELECTOR, 'a.next.page-numbers').click()
    time.sleep(3)
    
for url in urls:
    driver.get(url)
    time.sleep(1)
    section= driver.find_element(By.ID, "section-additional_information")
    try:
        op_sys = section.find_element(By.CSS_SELECTOR, "tr.woocommerce-product-attributes-item--attribute_pa_lap_operating-system-with > td")
        Operating_sys.append(op_sys.text)
    except NoSuchElementException:
        Operating_sys.append("-")
        
    try:
        display = section.find_element(By.CSS_SELECTOR, "tr.woocommerce-product-attributes-item--attribute_pa_lap_display-size-in-inche > td")
        Display.append(display.text)
    except NoSuchElementException:
        Display.append("-")
        
    try:
        processor = section.find_element(By.CSS_SELECTOR, "tr.woocommerce-product-attributes-item--attribute_pa_lap_processor-model-name > td")
        Processor.append(processor.text)
    except NoSuchElementException:
        Processor.append("-")
        
    try:
        memory = section.find_element(By.CSS_SELECTOR, "tr.woocommerce-product-attributes-item--attribute_pa_lap_operating-system-with > td")
        Memory.append(memory.text)
    except NoSuchElementException:
        Memory.append("-")
    try:
        storage_capacity = section.find_element(By.CSS_SELECTOR, "tr.woocommerce-product-attributes-item--attribute_pa_lap_storage-drive-capacity > td")
        Storage_capacity.append(storage_capacity.text)
    except NoSuchElementException:
        Storage_capacity.append("-")
    try:
        price = driver.find_element(By.CSS_SELECTOR, ".cegg-price.cegg-price-color.cegg-price-instock")
        Price.append(price.text)
    except NoSuchElementException:
        Price.append("-")
        
    try:
        name = driver.find_element(By.CSS_SELECTOR, ".woo_bl_title > h1")
        Laptop_Name.append(name.text)
    except NoSuchElementException:
        Laptop_Name.append("-")
        
DT=pd.DataFrame({
                'Laptop Name': Laptop_Name,
                'Operating System': Operating_sys,
                'Display': Display,
                'Processor': Processor,
                'Memory': Memory,
                'Storage Capacity': Storage_capacity,
                'Price': Price
                })
DT

,Laptop Name,Operating System,Display,Processor,Memory,Storage Capacity,Price
0,HP OMEN Gaming Laptop 16 XD0007ax,Windows 11 Home,16.1,AMD Ryzen 7-7840HS,Windows 11 Home,1 TB,"Rs. 101,990"
1,Dell G15 5520 Gaming Laptop,Windows 11 Home,15.6,12th Gen Intel Core i5-12500H,Windows 11 Home,512 GB,"Rs. 72,990"
2,Acer Predator Helios Neo 16 Gaming Laptop,Windows 11 Home,16,13th Gen Intel Core i7-13700HX,Windows 11 Home,512 GB,"Rs. 109,990"
3,HP OMEN Gaming Laptop (Xf0060AX),Windows 11 Home,16.1,AMD Ryzen 7-7840HS,Windows 11 Home,1 TB,"Rs. 117,490"
4,Dell Alienware M16 R1 Gaming Laptop,-,-,-,-,-,-
5,Dell Alienware X14 R2 Gaming Laptop,Windows 11 Home,14,13th Gen Intel Core i7-13620H,Windows 11 Home,1 TB,"Rs. 224,290"
6,Hp Victus Gaming Laptop 16-R0075TX,Windows 11 Home,16,13th Gen Intel Core i5-13500HX,Windows 11 Home,512 GB,-
7,Alienware X14 Gaming Laptop,-,-,-,-,-,-
8,ASUS TUF Gaming Laptop FX506LI-HN270T,-,-,-,-,-,-
9,Lenovo IdeaPad Gaming 3 11th Gen Core I5-11320...,-,-,-,-,-,-


9. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall
reviews, privates from price, dorms from price, facilities and property description

In [42]:
driver=webdriver.Chrome()
driver.get("https://www.hostelworld.com/")
search = driver.find_element(By.XPATH, "//div[@class='input-wrapper']/input")
search.send_keys("London")
time.sleep(2)
london= driver.find_element(By.XPATH, "//li[@aria-label='London']")
london.click()
search_btn= driver.find_element(By.CSS_SELECTOR, ".medium-button")
search_btn.click()
time.sleep(2)

hostel_name = []
distance = []
pvt_prices = []
dorms_price = []
rating = []
reviews = []
overall_reviews = []
facilities = []
description = []
urls = []


property_card = driver.find_elements(By.CSS_SELECTOR, "div:not(.featured-list) > .property-card")
for property_item in property_card:
    property_name = property_item.find_element(By.CLASS_NAME, "property-name")
    hostel_name.append(property_name.text)

    try:
        distance_ele = property_item.find_element(By.CSS_SELECTOR, ".distance-description")
        distance.append(distance_ele.text.replace('- ', ''))
    except NoSuchElementException:
        distance.append('-')

    try:
        prices = property_item.find_elements(By.CSS_SELECTOR, ".accommodation-price.single-row > .current")
        if len(prices) > 1:
            pvt_prices.append(prices[0].text)
            dorms_price.append(prices[1].text)
        else:
            pvt_prices.append('-')
            dorms_price.append(prices[0].text)
                
    except NoSuchElementException:
        pvt_prices.append('-')
        dorms_price.append('-')

    try:
        rating_el = property_item.find_element(By.CSS_SELECTOR, ".rating-score > .number")
        rating.append(rating_el.text)
    except NoSuchElementException:
        rating.append('-')
    try:
        reviews_property = property_item.find_element(By.CSS_SELECTOR, ".rating-score > .keyword")
        reviews.append(reviews_property.text)
    except NoSuchElementException:
        reviews.append('-')
    try:
        pattern=re.compile(r"[()]|[\)]")
        over_all = property_item.find_element(By.CLASS_NAME, "left-margin")
        match= pattern.sub('', over_all.text)
        overall_reviews.append(match)
    except NoSuchElementException:
        overall_reviews.append('-')
        
      
    url = property_item.find_element(By.CSS_SELECTOR, "a.property-card-container")
    urls.append(url.get_attribute('href'))
for url in urls:
    driver.get(url)
    time.sleep(1)
    try:
        description_el = driver.find_element(By.CSS_SELECTOR, ".description-container")
        description.append(description_el.text.replace('\n', ''))
    except NoSuchElementException:
            description.append('-')
            
DT=pd.DataFrame({
                'Hostel Name': hostel_name,
                'Distance': distance,
                'Private Prices': pvt_prices,
                'Dorms price': dorms_price,
                'Rating': rating,
                'Reviews': reviews,
                'Overall reviews' : overall_reviews,
                'Description': description
                
                })
DT




,Hostel Name,Distance,Private Prices,Dorms price,Rating,Reviews,Overall reviews,Description
0,Wombat's City Hostel London,3.6km from city centre,$257,$49,9.1,Superb,15918,Property DescriptionA safe haven in the middle...
1,Palmers Lodge - Swiss Cottage,6.5km from city centre,-,$34,8.8,Fabulous,15980,Hostelworld saysPalmers Lodge Swiss Cottage is...
2,St Christopher's Village,1.8km from city centre,$103,$20,7.8,Very Good,12878,Hostelworld saysHostelworld visitors have said...
3,Urbany Hostel London,5.4km from city centre,$161,$46,9.5,Superb,1127,Property DescriptionWelcome to Urbany Hostel L...
4,Onefam Notting Hill,5.5km from city centre,$138,$36,9.6,Superb,2603,Property DescriptionThe perfect place for solo...
5,Generator London,3km from city centre,$158,$46,7.5,Very Good,8087,Hostelworld saysHostelworld guests have said G...
6,Onefam Waterloo,0.7km from city centre,-,$55,9.6,Superb,146,Property DescriptionCome join us at our newest...
7,Astor Hyde Park,4.3km from city centre,-,$40,8.2,Fabulous,12331,Hostelworld saysNote: 18-40 yrs old only. Asto...
8,Safestay London Elephant & Castle,1.7km from city centre,-,$29,7.2,Very Good,5284,Hostelworld saysSafestay at Elephant and Castl...
9,Astor Museum Inn,2.2km from city centre,$120,$53,8.4,Fabulous,9457,Hostelworld saysNote: 18-40 yrs old only. Asto...
